In [ ]:
import os
import os, cv2
import numpy as np
import pandas as pd
import random, tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import segmentation_models_pytorch.utils as smp_utils
import warnings
warnings.filterwarnings("ignore")
from dataset import heartdataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as album
import segmentation_models_pytorch as smp
import utils
import config
from torchvision import models
# Set seed for reproducibility
utils.seed_everything(config.seed)


# Set paths to train, val and test images and masks
n_points = 30
w_mask = 2
 
path_for_segmentation_data_model=f'../../data/segmentation_data/segmentation_lines_{n_points}_{w_mask}' ## path for one of the trail,test,val folders to used in model 
x_test_dir=path_for_segmentation_data_model+'/test/images/'
y_test_dir=path_for_segmentation_data_model+'/test/masks/'


#x_test_dir = "/Users/giuliamonopoli/Desktop/PhD /deepvalve/data/segmentation_data/segmentation_lines_30_2/val/images/"
#y_test_dir = "/Users/giuliamonopoli/Desktop/PhD /deepvalve/data/segmentation_data/segmentation_lines_30_2/val/masks/"
#x_test_dir = "/home/ashay/deepvale/data/segmentation_data/segmentation_lines_30_2/val/images/"
#y_test_dir = "/home/ashay/deepvale/data/segmentation_data/segmentation_lines_30_2/val/masks/"



class_dict = pd.read_csv(config.labels)
# Get class names
class_names = class_dict['name'].tolist()
# Get class RGB values
class_rgb_values = class_dict[['r','g','b']].values.tolist()

# print('All dataset classes and their corresponding RGB values in labels:')
# print('Class Names: ', class_names)
# print('Class RGB values: ', class_rgb_values)

# Useful to shortlist specific classes in datasets with large number of classes
select_classes = ['background', 'leaflet']

# Get RGB values of required classes
select_class_indices = [class_names.index(cls.lower()) for cls in select_classes]
select_class_rgb_values =  np.array(class_rgb_values)[select_class_indices]

# print('Selected classes and their corresponding RGB values in labels:')
# print('Class Names: ', class_names)
# print('Class RGB values: ', class_rgb_values)

ENCODER = config.ENCODER
ENCODER_WEIGHTS = config.ENCODER_WEIGHTS
CLASSES = class_names
ACTIVATION = config.ACTIVATION # could be None for logits or 'softmax2d' for multiclass segmentation

# create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=len(CLASSES),
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
model = model.to("cpu")

: 

In [3]:
# load best saved model checkpoint from the current ru

import dataset
import utils
import config


model_path="/home/daniel/deepvalve/src/segmentation/segmentation_results_exploitation/sandy-sweep-8best_model.pth"

# if os.path.exists('../data/models/'):
model.load_state_dict(torch.load(model_path))
model.eval()

    

# load best saved model checkpoint from previous commit (if present)
# elif os.path.exists('../input//unet-with-pretrained-resnet50-encoder-pytorch/best_model.pth'):
#     best_model = torch.load('../input//unet-with-pretrained-resnet50-encoder-pytorch/best_model.pth', map_location=DEVICE)
#     print('Loaded UNet model from a previous commit.')
# create test dataloader (with preprocessing operation: to_tensor(...))
test_dataset = dataset.heartdataset(
    x_test_dir,
    y_test_dir,
    augmentation=utils.get_validation_augmentation(),
    preprocessing=utils.get_preprocessing(preprocessing_fn),
    class_rgb_values=select_class_rgb_values,
)

test_dataloader = DataLoader(test_dataset)

# test dataset for visualization (without preprocessing transformations)
test_dataset_vis = dataset.heartdataset(
    x_test_dir, y_test_dir,
    augmentation=utils.get_validation_augmentation(),
    class_rgb_values=select_class_rgb_values,
)

# get a random test image/mask index
random_idx = random.randint(0, len(test_dataset_vis)-1)
image, mask = test_dataset_vis[random_idx]

utils.visualize(
    original_image = image,
    ground_truth_mask = utils.colour_code_segmentation(utils.reverse_one_hot(mask), select_class_rgb_values),
    one_hot_encoded_mask = utils.reverse_one_hot(mask)
)
# Center crop padded image / mask to original image dims
def crop_image(image, target_image_dims=[448,448,3]):

    target_size = target_image_dims[0]
    image_size = len(image)
    padding = (image_size - target_size) // 2

    return image[
        padding:image_size - padding,
        padding:image_size - padding,
        :,
    ]

sample_preds_folder = 'sample_predictions/'
if not os.path.exists(sample_preds_folder):
    os.makedirs(sample_preds_folder)

RuntimeError: Error(s) in loading state_dict for Unet:
	Missing key(s) in state_dict: "encoder._conv_stem.weight", "encoder._bn0.weight", "encoder._bn0.bias", "encoder._bn0.running_mean", "encoder._bn0.running_var", "encoder._blocks.0._depthwise_conv.weight", "encoder._blocks.0._bn1.weight", "encoder._blocks.0._bn1.bias", "encoder._blocks.0._bn1.running_mean", "encoder._blocks.0._bn1.running_var", "encoder._blocks.0._se_reduce.weight", "encoder._blocks.0._se_reduce.bias", "encoder._blocks.0._se_expand.weight", "encoder._blocks.0._se_expand.bias", "encoder._blocks.0._project_conv.weight", "encoder._blocks.0._bn2.weight", "encoder._blocks.0._bn2.bias", "encoder._blocks.0._bn2.running_mean", "encoder._blocks.0._bn2.running_var", "encoder._blocks.1._depthwise_conv.weight", "encoder._blocks.1._bn1.weight", "encoder._blocks.1._bn1.bias", "encoder._blocks.1._bn1.running_mean", "encoder._blocks.1._bn1.running_var", "encoder._blocks.1._se_reduce.weight", "encoder._blocks.1._se_reduce.bias", "en

## Sample

In [ ]:
for idx in range(len(test_dataset)):

    image, gt_mask = test_dataset[idx]
    image_vis = crop_image(test_dataset_vis[idx][0].astype('uint8'))
    x_tensor = torch.from_numpy(image).to("cpu").unsqueeze(0)
    # Predict test image
    pred_mask = model(x_tensor)
    pred_mask = pred_mask.detach().squeeze().cpu().numpy()
    # Convert pred_mask from `CHW` format to `HWC` format
    pred_mask = np.transpose(pred_mask,(1,2,0))
    # Get prediction channel corresponding to building
    pred_building_heatmap = pred_mask[:,:,select_classes.index('leaflet')]
    pred_mask = crop_image(utils.colour_code_segmentation(utils.reverse_one_hot(pred_mask), select_class_rgb_values))
    # Convert gt_mask from `CHW` format to `HWC` format
    gt_mask = np.transpose(gt_mask,(1,2,0))
    gt_mask = crop_image(utils.colour_code_segmentation(utils.reverse_one_hot(gt_mask), select_class_rgb_values))

    ## to save them
    # cv2.imwrite(os.path.join(sample_preds_folder, f"sample_pred_{idx}.png"), np.hstack([image_vis, gt_mask, pred_mask])[:,:,::-1])  

    utils.visualize(
        original_image = image_vis,
        ground_truth_mask = gt_mask,
        predicted_mask = pred_mask,
        predicted_leaflet_heatmap = pred_building_heatmap
    )